In [ ]:


import requests
import json
import pandas as pd
import psycopg2
import paramiko
import sys
import numpy as np

# Function to create a database connection
def connect_db():
    con = psycopg2.connect(host='your-host', 
                           database='your-database',
                           user='your-user', 
                           password='your-password')
    return con

def query_db(sql):
    con = connect_db()
    cur = con.cursor()
    cur.execute(sql)
    recset = cur.fetchall()
    records = []
    for rec in recset:
        records.append(rec)
    con.close()
    return records

# Retrieve Quote table
bdquote = query_db(""" 
select distinct q.id_quote as "Quote Id",
        q.nr_quote as "Quote Number",
        q.id_opportunity as "Opportunity Id",
        q.id_vendedor as "Seller Id",
        q.id_cliente as "Client Id",
        q.id_order as "Order Id",
        q.id_especificador as "Specifier Id",
        q.ds_status as "Quote Status",
        q.dt_lastmodified::date as "Modification Date",
        q.dt_emissao as "Issuance Date",
        q.dt_prev_fechamento as "Expected Closing Date",
        q.fl_orcamento_principal as "Main Quote",
        q.dt_desejo as "Desired Date",
        q.dt_expiracao as "Expiration Date",
        q.cd_motivo_cancelamento as "Cancellation Reason Code",
        q.ds_motivo_cancelamento as "Cancellation Reason Description",
        dr.ds_name as "Sales Type",
        sq.id as "Quote Item Id",
        sq.nm_ambiente as "Environment",
        sq.fl_corte_especial as "Special Cut",
        ds_classificacao as "Product Classification",
        ds_formato as "Product Format",
        ds_grandes_formatos as "Large Format Products",
        ds_linha as "Product Line",
        nm_produto as "Product Name",
        ds_tipologia_cml as "Commercial Typology",
        ds_tipologia as "Main Typology",
        cd_unimed as "Unit of Measure",
        ds_fase_vida as "Life Stage",
        class_port_1 as "Product Category 1",
        class_port_2 as "Product Category 2",
        class_port_3 as "Product Category 3",
        class_port_4 as "Product Category 4"
  from your_schema.stage_quote q
  left join your_schema.stage_quotelineitem sq on sq.id_quote = q.id_quote
  left join your_schema.dim_product2 p on p.id = sq.id_product2
  left join your_schema.dim_produto_orc po on po.cod_produto_ora = p.id_productcode
  left join your_schema.dim_recordtype dr on dr.id = q.recordtype_id
""")

# Transform query results into DataFrame
bdquote = pd.DataFrame(bdquote, columns=["Quote Id", "Quote Number", "Opportunity Id", "Seller Id", "Client Id", "Order Id", "Specifier Id", "Quote Status",
"Modification Date", "Issuance Date", "Expected Closing Date", "Main Quote", "Desired Date", "Expiration Date",
"Cancellation Reason Code", "Cancellation Reason Description", "Sales Type", "Quote Item Id", "Environment", "Special Cut",
"Product Classification", "Product Format", "Large Format Products", "Product Line", "Product Name", "Commercial Typology",
"Main Typology", "Unit of Measure", "Life Stage", "Product Category 1", "Product Category 2", "Product Category 3",
"Product Category 4"])

bdquote['Issuance Date'] = pd.to_datetime(bdquote['Issuance Date'], utc=True)
bdquote['Year'] = bdquote['Issuance Date'].dt.year

# Group data for analysis
analysis_data = bdquote[['Quote Id', 'Quote Status', 'Year']].groupby(by=['Quote Status', "Year"]).agg({"Quote Id": [np.count_nonzero]})

# Export analysis result to Excel
analysis_data.to_excel('path_to_save\\quote_analysis.xlsx', index=False)

# Prepare order data for merging
order_data_prepared = bdquote[['Quote Id', 'Quote Status', 'Year']]

# Merge order data
bdorder3 = bdorder2.merge(bdquote, how='left', left_on='Quote Id', right_on='Quote Id') 
bdorder3.head()

# Convert order data to DataFrame
bdorder = pd.DataFrame(bdorder, columns=["Order Id", "Order Item Id", "Quote Item Id", "Order Number", "Cancellation Date", "Sale Date", "Quote Id",
"Seller Id", "Client Id", "Store Id", "Specifier Id", "Order Status", "Sales Type Order", "Product Classification",
"Product Format", "Large Format Products", "Product Line", "Product Name", "Commercial Typology", "Main Typology",
"Life Stage", "Product Category 1", "Product Category 2", "Product Category 3", "Product Category 4"])

# Export order data to CSV
bdorder.to_csv('path_to_save\\orders.csv', index=False)
